<h1></h1>
<p>From Novel Variants of PPP2R1A in Catalytic Subunit Binding Domain and Genotype–Phenotype Analysis in Neurodevelopmentally Delayed Patients -- apparnetly no pmid yet</p>
https://doi.org/10.3390/genes14091750

 Furthermore, we found that pathogenic variants clustered in HEAT repeats V, VI and VII, and patients with the Met180Val/Thr variants had macrocephaly, severe ID and hypotonia, but no epilepsy, whereas those with Arg258 amino acid changes had microcephaly, while a few had epilepsy or feeding problems.

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import *
import importlib.metadata
__version__ = importlib.metadata.version("pyphetools")
print(f"Using pyphetools version {__version__}")

Using pyphetools version 0.5.2


In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199")
metadata.default_versions_with_hpo(version=hpo_version)

In [115]:
df = pd.read_excel("input/qian_2023.xlsx")

In [116]:
df.head()

,Individual,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,Age/Sex,11y/F,4y/M,18y/M,18y/F,12y/M,4y/M,3.5y/F,5m/F,27y/M,...,?/M,?/M,?/M,?/F,?/F,?/F,?/F,?/F,?/F,?/F
1,Variant,c.96C>G,c.421T>A,c.455C>T,c.532A>T,c.533C>A,c.536C>T,c.536C>T,c.536C>A,c.538A>G,...,c.533C>G,c.533C>A,c.536C>T,c.544C>T,c.544C>T,c.544C>T,c.656C>T,c.658G>A,c.658G>A,c.772C>A
2,Inheritance,De novo,De novo,De novo,De novo,De novo,De novo,De novo,De novo,De novo,...,De novo,De novo,De novo,De novo,De novo,De novo,De novo,De novo,De novo,De novo
3,Birth size,N,N,N,/,/,N,N,N,N,...,/,/,/,/,/,/,/,/,/,/
4,Height,N,N,N,N,N,-3SD,N,N,N,...,/,/,/,/,/,/,/,/,/,/


In [117]:
# convert to row-based format
dft = df.transpose()
dft.columns = dft.iloc[0]
dft.drop(dft.index[0], inplace=True)
dft['patient_id'] = dft.index
dft.head()

Individual,Age/Sex,Variant,Inheritance,Birth size,Height,Head circumference,DD/ID,Language delay,Motor delay,Epilepsy,Brain MR,Behavior problem,Hypotonia,Feeding problem,Hearing loss,Extremities/spine,Heart,PMID,patient_id
1,11y/F,c.96C>G,De novo,N,N,N,"ID, moderate",moderate,-,Moderate,N,"ASD, self-injury",+,-,-,Hyperm.,N,PMID:33106617,1
2,4y/M,c.421T>A,De novo,N,N,macro,"ID, severe",Moderate,+ (walk 4y9m),-,N,ADHD,+,GR,-,Hyperm.,N,PMID:33106617,2
3,18y/M,c.455C>T,De novo,N,N,N,IQ 86,Mild,-(walk 15m),-,N,ASD,-,-,-,Hyperm.,N,PMID:33106617,3
4,18y/F,c.532A>T,De novo,/,N,+5.1SD,"ID, moderate",mild,+(walk 2y),-,/,anxious,+,-,-,"cubitus valgus, kyphoscoliosis, et al",/,PMID:33106617,4
5,12y/M,c.533C>A,De novo,/,N,+2SD,"ID, severe",Severe,+(walk 5y),-,N,"ADHD, destructive",+,+,+,"Hyperm. scoliosis, et al",/,PMID:33106617,5


In [118]:
column_mapper_d = defaultdict(ColumnMapper)
dft.columns

Index(['Age/Sex', 'Variant', 'Inheritance', 'Birth size', 'Height',
       'Head circumference', 'DD/ID', 'Language delay', 'Motor delay',
       'Epilepsy', 'Brain MR', 'Behavior problem', 'Hypotonia',
       'Feeding problem', 'Hearing loss', 'Extremities/spine', 'Heart', 'PMID',
       'patient_id'],
      dtype='object', name=' Individual ')

In [119]:
#output = OptionColumnMapper.autoformat(df=dft, concept_recognizer=hpo_cr)
# print(output)

In [120]:
# Intrauterine growth retardation HP:0001511
birth_size_d = { 'SGA': 'Intrauterine growth retardation',
 'small': 'Intrauterine growth retardation'}
excluded_d = {'N': 'Intrauterine growth retardation'}
birth_size_Mapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=birth_size_d, excluded_d=excluded_d)
birth_size_Mapper.preview_column(dft['Birth size'])
column_mapper_d['Birth size'] = birth_size_Mapper

In [121]:
height_d = { '-3SD': 'Short stature',
 '-2.6SD': 'Short stature',
 '-2.5SD': 'Short stature',
 'SS': 'Short stature'}
excluded_d = {'N': 'Short stature', '+2SD': 'Short stature',}
heightMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=height_d, excluded_d=excluded_d)
heightMapper.preview_column(dft['Height'])
column_mapper_d['Height'] = heightMapper

In [122]:

head_circumference_d = {
 'macro': 'Macrocephaly',
 '+5.1SD': 'Macrocephaly',
 'micro': 'Microcephaly',
 '+3SD': 'Macrocephaly',
 '+5.2SD': 'Macrocephaly',
 '+3.4SD': 'Macrocephaly',
 '+2.5SD': 'Macrocephaly',
 '+3.9SD': 'Macrocephaly',
 '-2.4SD': 'Microcephaly',
 'Micro': 'Microcephaly',
 '-2.5SD': 'Microcephaly',
 '-3.1SD': 'Microcephaly',
 '-3.5SD': 'Microcephaly',
 '-3SD': 'Microcephaly'}

excluded_d = {'N': 'Abnormality of skull size',}
head_circumferenceMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=head_circumference_d, excluded_d=excluded_d)
head_circumferenceMapper.preview_column(dft['Head circumference'])
column_mapper_d['Head circumference'] = head_circumferenceMapper

In [123]:
# dd/id, code as Global developmental delay
dd_d = {'ID': 'Intellectual disability',
 'moderate': 'Intellectual disability, moderate',
 'severe': 'Intellectual disability, severe',
 'ID/DD': 'Global developmental delay',
 'DD': 'Global developmental delay',
 'mild': 'Intellectual disability, mild',
}
ddMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=dd_d)
ddMapper.preview_column(dft['DD/ID'])
column_mapper_d['DD/ID'] = ddMapper

In [124]:
language_delay_d = {'moderate': 'Delayed speech and language development',
 'Moderate': 'Delayed speech and language development',
 'Mild': 'Delayed speech and language development',
 'mild': 'Delayed speech and language development',
 'Severe': 'Delayed speech and language development',
 'No words': 'Absent speech',
 'severe': 'Delayed speech and language development',
 '+': 'Delayed speech and language development',
 '/ ( no word)': 'Absent speech',
 '+ (30 word)': 'Delayed speech and language development',
 '+ (No word)': 'Absent speech'}
language_delayMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=language_delay_d)
language_delayMapper.preview_column(dft['Language delay'])
column_mapper_d['Language delay'] = language_delayMapper

In [125]:
motor_delay_d = {
 '+ (walk 4y9m)': 'Motor delay',

 '+(walk 2y)': 'Motor delay',
 '+(walk 5y)': 'Motor delay',
 '+ (can’t walk)': 'Motor delay',
 '+ (walk 2y)': 'Motor delay',
 '+ (walk 3y)': 'Motor delay',
 '+ (walk 2.5y)': 'Motor delay',
 '+(can’t sit 4.5y)': 'Motor delay',
 '+ (walk 6y)': 'Motor delay',
 '/ (no walk)': 'Motor delay',
 '+': 'Motor delay',
 '+(walk 2.5y)': 'Motor delay',
 '+ (walk 23m)': 'Motor delay',
 '+(walk 3y)': 'Motor delay',
 '+ (can’t crawl/sit)': 'Motor delay'}
excluded_d = {'-': 'Motor delay', '-(walk 15m)': 'Motor delay',}
motor_delayMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=motor_delay_d, excluded_d=excluded_d)
motor_delayMapper.preview_column(dft['Motor delay'])
column_mapper_d['Motor delay'] = motor_delayMapper

In [126]:
epilepsy_d = {'Moderate': 'Seizure',
 'moderate': 'Seizure',
 '/': 'PLACEHOLDER',
 '+': 'Seizure',
 'Mild': 'Seizure',
 'severe': 'Seizure',
 '/(Once febrile seizure)': 'Febrile seizure (within the age range of 3 months to 6 years)'}
excluded_d = {'-': 'Seizure'}
epilepsyMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=epilepsy_d, excluded_d=excluded_d)
epilepsyMapper.preview_column(dft['Epilepsy'])
column_mapper_d['Epilepsy'] = epilepsyMapper

In [127]:
brain_mr_d = {
 'CCA': 'Agenesis of corpus callosum',
 'ventriculomegaly': 'Ventriculomegaly',
 'white matter diffuse thinning': 'Reduced cerebral white matter volume',
 'PVLM': 'Periventricular leukomalacia',
 'hippocampal dysplasia': 'Dysgenesis of the hippocampus',
 'CCH': 'Hypoplasia of the corpus callosum',
 'brain stem hypoplasia': 'Hypoplasia of the brainstem',
 'hydrocephalus': 'Hydrocephalus',
 'pachygyria et al': 'Pachygyria',
 'bilateral ventriculomegaly': 'Ventriculomegaly',
 'enlarged third ventricle': 'Dilated third ventricle',
 'hypoplastic brainstem': 'Hypoplasia of the brainstem',
 'Partial CCA': 'Partial agenesis of the corpus callosum',
 'lateral ventriculomegaly': 'Lateral ventricle dilatation'}
brain_mrMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=brain_mr_d)
brain_mrMapper.preview_column(dft['Brain MR'])
column_mapper_d['Brain MR'] = brain_mrMapper

In [128]:
behavior_problem_d = {'ASD': 'Autistic behavior',
 'self-injury': 'Self-injurious behavior',
 'ADHD': 'Attention deficit hyperactivity disorder',
 'anxious': 'Anxiety',
 #'destructive': 'PLACEHOLDER',
 #'Hand chews': 'PLACEHOLDER',
 'aggressive': 'Aggressive behavior',
 'stereotypic': 'Abnormal repetitive mannerisms',
 'Self-injury': 'Self-injurious behavior',
 'hyperactivity': 'Hyperactivity',
 'autistic': 'Autistic behavior',
 'stereotypy': 'Abnormal repetitive mannerisms',
 'Autistic behavior': 'Autistic behavior'}
excluded_d = { '-': 'Atypical behavior',}
behavior_problemMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=behavior_problem_d)
behavior_problemMapper.preview_column(dft['Behavior problem'])
column_mapper_d['Behavior problem'] = behavior_problemMapper

In [129]:
hypotonia_d = {'+': 'Hypotonia',
 'Moderate': 'Hypotonia',
 'severe': 'Hypotonia',
 'moderate': 'Hypotonia',
}
excluded_d = { '-': 'Hypotonia'}
hypotoniaMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=hypotonia_d)
hypotoniaMapper.preview_column(dft['Hypotonia'])
column_mapper_d['Hypotonia'] = hypotoniaMapper

In [130]:
feeding_problem_d = {'-': 'Feeding difficulties',
 'GR': 'Gastroesophageal reflux',
 '+': 'Feeding difficulties',
 'TPN': 'Dependency on parenteral nutrition',
 'G-tube': 'Tube feeding',
 'mild': 'Feeding difficulties',
 'PEG': 'Tube feeding'}
excluded_d = { '-': 'Feeding difficulties'}
feeding_problemMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=feeding_problem_d, excluded_d=excluded_d)
feeding_problemMapper.preview_column(dft['Feeding problem'])
column_mapper_d['Feeding problem'] = feeding_problemMapper

In [131]:
hearing_loss_d = {
 '+': 'Hearing impairment'}
excluded_d = {'-': 'Hearing impairment'}
hearing_lossMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=hearing_loss_d, excluded_d=excluded_d)
hearing_lossMapper.preview_column(dft['Hearing loss'])
column_mapper_d['Hearing loss'] = hearing_lossMapper

In [132]:
extremities_spine_d = {'Hyperm.': 'Joint hypermobility',
 'cubitus valgus': 'Cubitus valgus',
 'kyphoscoliosis': 'Kyphoscoliosis',
 'Hyperm. scoliosis': 'Scoliosis',
 'hip & knee dislocations': 'Knee dislocation',
 'thoracolumbar scoliosis': 'Thoracolumbar scoliosis',
 'scoliosis': 'Scoliosis',
 'Genu recurvatum': 'Genu recurvatum',
 'Scoliosis': 'Scoliosis',
 'hyperm.': 'Joint hypermobility'}
extremities_spineMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=extremities_spine_d)
extremities_spineMapper.preview_column(dft['Extremities/spine'])
column_mapper_d['Extremities/spine'] = extremities_spineMapper

In [133]:
heart_d = {
 'PDA': 'Patent ductus arteriosus',
 'ASD': 'Atrial septal defect',
 'PFO': 'Patent foramen ovale',
 'asymmetric LVH': 'Left ventricular hypertrophy',
 'PS': 'Pulmonic stenosis',
 'VSD': 'Ventricular septal defect',
 'BAV': 'Bicuspid aortic valve',
 'DAA': 'Ascending tubular aorta aneurysm'}
heartMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=heart_d)
heartMapper.preview_column(dft['Heart'])
column_mapper_d['Heart'] = heartMapper

In [134]:
pmid_d = {'33106617': 'PLACEHOLDER',
 '26168268': 'PLACEHOLDER',
 '36209351': 'PLACEHOLDER',
 '25533962': 'PLACEHOLDER',
 '31687265': 'PLACEHOLDER',
 '31531803': 'PLACEHOLDER',
 '30755392': 'PLACEHOLDER',
 'Chinese paper': 'PLACEHOLDER',
 '34716204': 'PLACEHOLDER',
 'VCV001064777': 'PLACEHOLDER',
 'VCV000190312': 'PLACEHOLDER',
 'VCV000376505': 'PLACEHOLDER',
 'VCV000217458': 'PLACEHOLDER',
 'DECIPHER:274685': 'PLACEHOLDER',
 'DECIPHER:275646': 'PLACEHOLDER',
 'DECIPHER:457603': 'PLACEHOLDER',
 'DECIPHER:268550': 'PLACEHOLDER',
 'DECIPHER:259358': 'PLACEHOLDER',
 'DECIPHER:258589': 'PLACEHOLDER',
 'DECIPHER:263907': 'PLACEHOLDER',
 'DECIPHER:304649': 'PLACEHOLDER',
 'DECIPHER:428704': 'PLACEHOLDER',
 'DECIPHER:280097': 'PLACEHOLDER',
 'DECIPHER:379691': 'PLACEHOLDER',
 'DECIPHER:286623': 'PLACEHOLDER'}
#pmid Mapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=pmid _d)
#pmid Mapper.preview_column(df['PMID '])
#column_mapper_d['PMID '] = pmid Mapper

In [135]:
dft['Variant']

1       c.96C>G
2      c.421T>A
3      c.455C>T
4      c.532A>T
5      c.533C>A
6      c.536C>T
7      c.536C>T
8      c.536C>A
9      c.538A>G
10     c.538A>G
11     c.538A>G
12     c.539T>C
13     c.539T>C
14     c.539T>C
15     c.539T>C
16     c.539T>C
17     c.539T>A
18     c.539T>G
19     c.544C>T
20     c.544C>T
21     c.544C>T
22     c.544C>T
23     c.544C>T
24     c.544C>T
25     c.544C>T
26     c.547C>T
27     c.548G>A
28     c.548G>A
29     c.548G>A
30     c.656C>T
31     c.656C>T
32     c.656C>T
33     c.656C>T
34     c.658G>A
35     c.658G>A
36     c.658G>A
37     c.658G>A
38     c.772C>A
39     c.773G>A
40     c.773G>A
41     c.773G>A
42     c.773G>A
43     c.773G>A
44     c.773G>A
45     c.536C>A
46     c.544C>T
47     c.547C>T
48     c.773G>A
49       c.2T>C
50       c.2T>C
51     c.533C>G
52     c.533C>A
53     c.536C>T
54     c.544C>T
55     c.544C>T
56     c.544C>T
57     c.656C>T
58     c.658G>A
59     c.658G>A
60     c.772C>A
Name: Variant, dtype: object

In [143]:
dft['Age'] = dft['Age/Sex'].apply(lambda x: x.split("/")[0])
#dft['Sex'] = dft['Age/Sex'].apply(lambda x: {y=x.split("/"); if len(y)==1:return y[0];else:return y[1]})

<h2>Create separate columns for age and sex</h2>

In [144]:
dft['Age/Sex']

1          11y/F 
2           4y/M 
3          18y/M 
4          18y/F 
5          12y/M 
6           4y/M 
7         3.5y/F 
8           5m/F 
9          27y/M 
10         20y/M 
11         16m/M 
12        3y9m/M 
13          6y/F 
14          2y/F 
15         23y/F 
16          2y/M 
17         10m/F 
18          9y/F 
19          6y/F 
20          4y/M 
21          3y/F 
22         7m, F 
23          4y/F 
24         11y/M 
25          1y/F 
26          2y/M 
27     Neonate/M 
28     Neonate/M 
29     Neonate/M 
30         20y/M 
31        2y4m/M 
32          7y/M 
33          9m/M 
34          4y/F 
35          7y/F 
36          3y/M 
37          4y/M 
38         13m/M 
39          5y/M 
40          3y/M 
41          4y/M 
42        1y6m/M 
43          2y/M 
44         16m/M 
45             / 
46             / 
47             / 
48             / 
49           ?/F 
50           ?/M 
51           ?/M 
52           ?/M 
53           ?/M 
54           ?/F 
55           ?/F 
56        